In [2]:
!pip install -U langchain langchain-community langchain-text-splitters chromadb


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -U langchain langchain-community langchain-text-splitters chromadb


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [5]:
llm = Ollama(model="smollm2:360m")   # or llama3.2:1b

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

C:\Users\Sanidhya\AppData\Local\Temp\ipykernel_14100\1112943999.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="smollm2:360m")   # or llama3.2:1b
C:\Users\Sanidhya\AppData\Local\Temp\ipykernel_14100\1112943999.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [8]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [17]:
llm = Ollama(model="smollm2:360m")   # or llama3.2:1b

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

In [20]:
import os
print(os.path.exists(r"C:\Users\Sanidhya\OneDrive\Desktop\sdp 6sem\data.txt"))

True


In [21]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader(
    r"C:\Users\Sanidhya\OneDrive\Desktop\sdp 6sem\data.txt",
    encoding="utf-8"
)

docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

splits = splitter.split_documents(docs)

print("Number of chunks:", len(splits))

Number of chunks: 1


In [22]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="chroma_db"
)

vectorstore.persist()

C:\Users\Sanidhya\AppData\Local\Temp\ipykernel_14100\3535613377.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [23]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="smollm2:360m")

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template("""
Answer the question using only the context.

Context:
{context}

Question:
{question}
""")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke("What is AI?")
print(response)

AI (Artificial Intelligence) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning from experience, understanding complex concepts, and adapting to new information.


In [25]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(model="smollm2:360m")

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 6}  # fetch more chunks for packing
)

In [26]:
rephrase_prompt = ChatPromptTemplate.from_template("""
Rewrite the user question to make it more detailed and retrieval-friendly.

Original Question:
{question}

Rewritten Question:
""")

rephrase_chain = rephrase_prompt | llm | StrOutputParser()

In [27]:
def pack_context(docs, max_chars=2000):
    packed = ""
    for doc in docs:
        if len(packed) + len(doc.page_content) > max_chars:
            break
        packed += doc.page_content + "\n\n"
    return packed

In [28]:
rag_prompt = ChatPromptTemplate.from_template("""
You are a helpful AI assistant.

Use ONLY the context below to answer the question.
If answer is not in context, say "Not found in document."

Context:
{context}

Question:
{question}

Answer:
""")

rag_chain = rag_prompt | llm | StrOutputParser()

In [29]:
def ask_rag(question):
    
    # 1️⃣ Rewrite query
    rewritten_question = rephrase_chain.invoke({"question": question})
    
    # 2️⃣ Retrieve relevant chunks
    docs = retriever.invoke(rewritten_question)
    
    # 3️⃣ Pack context smartly
    context = pack_context(docs)
    
    # 4️⃣ Generate final answer
    answer = rag_chain.invoke({
        "context": context,
        "question": question
    })
    
    return answer

In [30]:
response = ask_rag("What is this document about?")
print(response)

This document talks about a popular online video series called GTYTYGVGYFYFYUGBHIH, where users can watch and learn from experienced actors in the comedy genre.
